In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [3]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["round summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Offer your assitance and then introduce yourself casually
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 1. TURN: 1 


Rushing to her side, you quickly assess the situation before offering your assistance. "Hey, are you okay? That looks painful," you say with genuine concern in your voice. After helping her to a safer spot, you introduce yourself, "I'm Hikaru, by the way. I was just hiking around here. Can I help you with that injury?"

Mitsuha looks up at you, surprise and relief washing over her features. "Oh, thank you so much! I'm Mitsuha. I was trying to take a shortcut and slipped. I think I twisted my ankle," she explains, attempting to stand but winces in pain.

You notice the injury seems minor but definitely needs some attention. "Let's get that ankle looked at. Do you need support walking? The hotel I'm staying at isn't too far from here; they might have a first-aid kit," you suggest, ready to lend a shoulder for her to lean on.

Mitsuha nods gratefully, "That would be amazing. Thank you, Hikaru. I'm actually staying at the same hotel, I think. What a coincidenc

I select option 3. TURN: 2 


With a light-hearted tone, you respond, "Well, since we're both staying at the same hotel and I'm somewhat new to the area, you can make it up to me by showing me around. I could use a good tour guide."

Mitsuha's eyes light up at the suggestion, and she chuckles, "That sounds like a fair deal! I'd be happy to show you around once this ankle gets better. There are some beautiful spots around here that are definitely worth visiting."

The hotel owner finishes bandaging Mitsuha's ankle and advises her to rest it for the evening. "You should be good as new by tomorrow if you take it easy tonight," she says with a warm smile.

Mitsuha nods and turns to you, "Looks like our tour will have to wait until tomorrow. But, how about dinner tonight? The hotel's restaurant has some amazing local dishes. It's the least I can do to thank you for your help today."

You're pleasantly surprised by her offer and the chance to spend more time with her. "Dinner sounds great," 